In [2]:
import numpy as np
from math import lgamma

## Read data

In [3]:
## Data file should be strings of 0s and 1s without spacing
def read_data(filename):
    Kset = {}
    N = 0
    with open(filename, 'r') as file:
        for line in file.readlines():
            s = int(line[:n],2)
            N += 1
            if s in Kset:
                Kset[s] += 1
            else:
                Kset[s] = 1
    return Kset, N

In [4]:
def print_partition(MCM):
    for com,ICC in MCM.items():
        binary = bin(ICC).replace("0b", "")
        missing = n - len(binary)
        print(f'com {com}\t' + '0'*missing + binary)

## Calculate log evidence

In [5]:
def complexity(m):
    power = 1 << (m-1)
    return np.log(np.pi) * power - np.math.lgamma(power)

def project_icc(Kset, ICC):
    new_Kset = {}
    for s,ks in Kset.items():
        s_new = s & ICC
        if s_new in new_Kset:
            new_Kset[s_new] += ks
        else:
            new_Kset[s_new] = ks
    return new_Kset, bin(ICC).count("1")

def log_evidence_ICC(Kset, ICC):
    LogE = 0
    new_Kset, nNode = project_icc(Kset, ICC)
    for s,ks in new_Kset.items():
        LogE += np.math.lgamma(ks + 0.5)
    LogE -= (complexity(nNode) + np.math.lgamma(N + (1 << (nNode - 1))))
    return LogE

def log_evidence_MCM(Kset, MCM):
    LogE = 0
    for _,ICC in MCM.items():
        LogE += log_evidence_ICC(Kset, ICC)
    return LogE

## Hierarchical merging

In [6]:
def merging(Kset):
    MCM = {}
    logE_MCM = {}
    
    for i in range(n):
        MCM[i] = (1 << i)
        logE_MCM[i] = log_evidence_ICC(Kset, MCM[i])
    
    logE_mat = np.zeros((n,n))
    
    stop = False
    
    while not stop and len(MCM)-1:
        best_diff = 0
        for i,node1 in MCM.items():
            for j,node2 in MCM.items():
                if i>=j:
                    continue

                if not logE_mat[i,j]:
                    logE_mat[i,j] = log_evidence_ICC(Kset, node1 + node2)
                    logE_mat[j,i] = logE_mat[i,j]
                
                if logE_mat[i,j] - logE_MCM[i] - logE_MCM[j] > best_diff:
                    best_diff = logE_mat[i,j] - logE_MCM[i] - logE_MCM[j]
                    best_merger = [[i,node1], [j,node2]]
                    
        if best_diff == 0:
            stop = True
            continue
        
        i_keep = best_merger[0][0]; node_keep = best_merger[0][1]
        i_erase = best_merger[1][0]; node_erase = best_merger[1][1]
                
        logE_MCM[i_keep] = logE_mat[i_keep, i_erase] 
        MCM[i_keep] = node_keep + node_erase
        
        logE_MCM.pop(i_erase)
        MCM.pop(i_erase)
        
        logE_mat[i_keep,:] = 0
        logE_mat[:,i_keep] = 0
        
    return MCM, sum([items for _,items in logE_MCM.items()])

## Measures

In [7]:
def norm_mut_info(MCM1, MCM2):
    I = 0
    H = 0
    flag = 0
    for _,ICC1 in MCM1.items():
        p1 = bin(ICC1).count("1") / n
        for _,ICC2 in MCM2.items():
            p2 = bin(ICC2).count("1") / n
            p12 = bin(ICC1 & ICC2).count("1") / n
            if p12:
                I += p12 * np.log(p12/(p1*p2))
            if flag < len(MCM2):
                H += p2 * np.log(p2)
                flag += 1
        H += p1 * np.log(p1)
    
    if not H:
        return 1
    else:
        return -2*I/H

In [8]:
def var_of_info(MCM1, MCM2):
    I = 0
    H = 0
    for _,ICC1 in MCM1.items():
        p1 = bin(ICC1).count("1") / n
        for _,ICC2 in MCM2.items():
            p2 = bin(ICC2).count("1") / n
            p12 = bin(ICC1 & ICC2).count("1") / n
            if p12:
                I += p12 * np.log(p12/(p1*p2))
                H += p12 * np.log(p12)
    
    if not H:
        return 0
    else:
        return 1 + I/H

## Tests

In [9]:
n = 20 ## specify number of nodes
filename = 'benchmark.dat' ## Give filename or path to file if not in the same folder

Kset, N = read_data(filename)

## MCM is a dictionary where the items are integers that represent communities when converted to binary.
## n = 4: 11 ~ 1101, 2 ~ 0010. This would imply that node 1,2,4 are in the same community and that node 3 is a community
## by itself.
MCM, logE = merging(Kset)

print(f'log-evidence: {logE:.2f}')
print_partition(MCM)

log-evidence: -121743.93
com 0	00000000000111000101
com 1	00100100000000101010
com 4	01000010110000010000
com 9	10011001001000000000


In [10]:
big = {0 : MCM[0] + MCM[1], 1 : MCM[4] + MCM[9]}

print(var_of_info(MCM,big))
print(norm_mut_info(MCM,big))

0.5
0.6666666666666667
